<a href="https://colab.research.google.com/github/otnemrasordep/DadaGP-track-processing/blob/main/bandGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/bandGP


/content/drive/MyDrive/bandGP


In [14]:
!ls

filtered_dg-dg-b      filtered_drums.zip	       tokenized_train_t5_dgdgb-drums
filtered_dg-dg-b.zip  __MACOSX			       tokenized_validation_t5_dgdgb-drums
filtered_drums	      paired-dataset-dgdgb-drums.json  wandb


In [4]:
# Install necessary libraries
!pip install transformers datasets wandb accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
# Import required libraries
import wandb
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset

In [ ]:
`f4b59c6bd3b9fe56f0dce11ecd227554c36bae87`

In [5]:
# Initialize Weights & Biases
# wandb.login(relogin=True)
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
# Start a new W&B run with increased timeout
wandb.init(
    project="bandGP-finetuning-t5",
    entity="otnemrasordep-queen-mary-university-of-london",
    settings=wandb.Settings(init_timeout=120),  # Increase timeout to 120 seconds
)

wandb: Currently logged in as: otnemrasordep (otnemrasordep-queen-mary-university-of-london). Use `wandb login --relogin` to force relogin


In [11]:
# Extract INPUT
import zipfile
import os
from tqdm.notebook import tqdm

# Path to your ZIP file
zip_file_path = "/content/drive/MyDrive/bandGP/filtered_dg-dg-b.zip"
# Destination folder for extracted files
extract_to = "/content/drive/MyDrive/bandGP/"

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Get the list of files in the ZIP archive
    file_list = zip_ref.namelist()
    total_files = len(file_list)

    # Use tqdm to show progress
    with tqdm(total=total_files, desc="Extracting files") as pbar:
        for file in file_list:
            zip_ref.extract(file, extract_to)
            pbar.update(1)  # Update progress


Extracting files:   0%|          | 0/49574 [00:00<?, ?it/s]

In [15]:
len(os.listdir('/content/drive/MyDrive/bandGP/filtered_dg-dg-b'))

49572

In [17]:
!rm -r /content/drive/MyDrive/bandGP/filtered_drums

In [18]:
# Extract OUTPUT
import zipfile
import os
from tqdm.notebook import tqdm

# Path to your ZIP file
zip_file_path = "/content/drive/MyDrive/bandGP/filtered_drums.zip"
# Destination folder for extracted files
extract_to = "/content/drive/MyDrive/bandGP/"

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Get the list of files in the ZIP archive
    file_list = zip_ref.namelist()
    total_files = len(file_list)

    # Use tqdm to show progress
    with tqdm(total=total_files, desc="Extracting files") as pbar:
        for file in file_list:
            # print(file)
            zip_ref.extract(file, extract_to)
            pbar.update(1)  # Update progress

Extracting files:   0%|          | 0/49574 [00:00<?, ?it/s]

In [19]:
len(os.listdir('/content/drive/MyDrive/bandGP/filtered_drums'))

49572

In [21]:
import json

# Collect input-output pairs
# T5 expects a "Input: xxx; Output: yyy" format for every file
pairs = []

# Input and output directories
input_dir = "/content/drive/MyDrive/bandGP/filtered_dg-dg-b"
output_dir = "/content/drive/MyDrive/bandGP/filtered_drums"

# Ensure files are sorted for pairing
input_files = sorted(os.listdir(input_dir))
output_files = sorted(os.listdir(output_dir))

# Create pairs
for x_file, y_file in tqdm(zip(input_files, output_files)):
    x_path = os.path.join(input_dir, x_file)
    y_path = os.path.join(output_dir, y_file)

    with open(x_path, "r") as x, open(y_path, "r") as y:
        input_text = x.read().strip()
        output_text = y.read().strip()
        pairs.append(f"Input: {input_text}\nOutput: {output_text}")

with open("paired-dataset-dgdgb-drums.json", "w") as final:
	json.dump(pairs, final)

0it [00:00, ?it/s]

In [8]:
import json

with open("paired-dataset-dgdgb-drums.json", "r") as final:
	pairs = json.load(final)

In [30]:
from datasets import load_dataset

# Load the JSON dataset
dataset = load_dataset("json", data_files={"data": "paired-dataset-dgdgb-drums.json"})
dataset = dataset["data"]  # Access the dataset


Generating data split: 0 examples [00:00, ? examples/s]

In [31]:
# Split into training and validation (80-20 split)
split = dataset.train_test_split(test_size=0.2, seed=42)

train_dataset = split["train"]
validation_dataset = split["test"]


In [32]:
print(f'Training set: {len(train_dataset)} exampples')
print(f'Validation set: {len(validation_dataset)} exampples')

Training set: 39657 exampples
Validation set: 9915 exampples


In [29]:
print(dataset[0])

{'text': 'Input: artist:dgdgb\ndowntune:-3\ntempo:100\nstart\nnew_measure\ndistorted0:note:s6:f-2\ndistorted1:note:s6:f-2\nbass:note:s5:f-2\nwait:480\ndistorted0:note:s3:f10\ndistorted0:note:s4:f10\ndistorted0:note:s5:f8\ndistorted1:note:s3:f10\ndistorted1:note:s4:f10\ndistorted1:note:s5:f8\nbass:note:s4:f8\nwait:240\ndistorted0:note:s3:f10\ndistorted0:note:s4:f10\ndistorted0:note:s5:f8\ndistorted1:note:s3:f10\ndistorted1:note:s4:f10\ndistorted1:note:s5:f8\nbass:note:s4:f8\nwait:240\ndistorted0:note:s3:f10\ndistorted0:note:s4:f10\ndistorted0:note:s5:f8\ndistorted1:note:s3:f10\ndistorted1:note:s4:f10\ndistorted1:note:s5:f8\nbass:note:s4:f8\nwait:240\ndistorted0:note:s3:f10\ndistorted0:note:s4:f10\ndistorted0:note:s5:f8\ndistorted1:note:s3:f10\ndistorted1:note:s4:f10\ndistorted1:note:s5:f8\nbass:note:s4:f8\nwait:240\ndistorted0:note:s5:f0\nnfx:dead\ndistorted0:note:s6:f-2\nnfx:dead\ndistorted1:note:s5:f0\nnfx:dead\ndistorted1:note:s6:f-2\nnfx:dead\nbass:rest\nwait:240\ndistorted0:note:s5

## Tokenization for T5


In [13]:
from transformers import AutoTokenizer

# Load the T5 tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")  # Replace with your desired T5 model size


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [35]:
def preprocess_text_split(example):
    text = example["text"]
    if "Input:" in text and "Output:" in text:
        input_start = text.find("Input:") + len("Input:")
        output_start = text.find("Output:")
        input_text = text[input_start:output_start].strip()
        target_text = text[output_start + len("Output:"):].strip()
    else:
        input_text = ""
        target_text = ""
    return {"input_text": input_text, "target_text": target_text}

# Apply preprocessing
train_dataset = train_dataset.map(preprocess_text_split)
validation_dataset = validation_dataset.map(preprocess_text_split)


Map:   0%|          | 0/9915 [00:00<?, ? examples/s]

In [36]:
validation_dataset[0]

{'text': 'Input: artist:dgdgb\ndowntune:-1\ntempo:130\nstart\nnew_measure\ndistorted0:rest\ndistorted1:rest\nbass:rest\nwait:1920\ndistorted0:rest\ndistorted1:rest\nwait:960\ndistorted0:rest\ndistorted1:rest\nwait:480\ndistorted0:note:s1:f7\ndistorted0:note:s2:f7\ndistorted0:note:s3:f8\ndistorted0:note:s4:f9\ndistorted0:note:s5:f9\ndistorted0:note:s6:f7\ndistorted1:note:s2:f12\ndistorted1:note:s3:f11\ndistorted1:note:s4:f9\nwait:240\ndistorted0:note:s1:f7\ndistorted0:note:s2:f7\ndistorted0:note:s3:f8\ndistorted0:note:s4:f9\ndistorted0:note:s5:f9\ndistorted0:note:s6:f7\ndistorted1:note:s2:f12\ndistorted1:note:s3:f11\ndistorted1:note:s4:f9\nwait:240\nnew_measure\ndistorted0:note:s1:f7\ndistorted0:note:s2:f7\ndistorted0:note:s3:f8\ndistorted0:note:s4:f9\ndistorted0:note:s5:f9\ndistorted0:note:s6:f7\ndistorted1:note:s2:f12\ndistorted1:note:s3:f11\ndistorted1:note:s4:f9\nbass:note:s4:f2\nwait:960\ndistorted0:note:s5:f11\ndistorted0:note:s6:f7\nbass:note:s4:f2\nwait:480\ndistorted0:note:s5:f

In [12]:
from transformers import T5Tokenizer

# Load T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Tokenization function
def preprocess_function(examples):
    # Tokenize input_text
    model_inputs = tokenizer(
        examples["input_text"],
        max_length=512,
        padding="max_length",
        truncation=True,
    )
    # Tokenize target_text as labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["target_text"],
            max_length=512,
            padding="max_length",
            truncation=True,
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [43]:
# Apply tokenization
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_validation = validation_dataset.map(preprocess_function, batched=True)


In [10]:
print(tokenized_validation)

Dataset({
    features: ['text', 'input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 9915
})


In [44]:
tokenized_train.save_to_disk("/content/drive/MyDrive/bandGP/tokenized_train_t5_dgdgb-drums")
tokenized_validation.save_to_disk("/content/drive/MyDrive/bandGP/tokenized_validation_t5_dgdgb-drums")

Saving the dataset (0/4 shards):   0%|          | 0/39657 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9915 [00:00<?, ? examples/s]

In [7]:
# To load tokenized datasets
from datasets import load_from_disk

tokenized_train = load_from_disk("/content/drive/MyDrive/bandGP/tokenized_train_t5_dgdgb-drums")
tokenized_validation = load_from_disk("/content/drive/MyDrive/bandGP/tokenized_validation_t5_dgdgb-drums")

In [8]:
import numpy as np

# Calculate token length statistics // TRAINING
token_lengths = [len(tokens["input_ids"]) for tokens in tokenized_train]
print(f"Max Length: {max(token_lengths)}")
print(f"Min Length: {min(token_lengths)}")
print(f"Median Length: {np.median(token_lengths)}")

# IDEAL FOR TRAINING!

Max Length: 512
Min Length: 512
Median Length: 512.0


In [11]:
# Remove unnecessary columns for fine-tuning
tokenized_train = tokenized_train.remove_columns(["text", "input_text", "target_text"])
tokenized_validation = tokenized_validation.remove_columns(["text", "input_text", "target_text"])

print(tokenized_train)
print(tokenized_validation)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 39657
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9915
})


## Training

In [1]:
# sanity check GPU
import torch

# Check if GPU is available
if torch.cuda.is_available():
    print(f"GPU is available: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available, using CPU instead.")

GPU is available: Tesla T4


In [17]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq

# Load the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Data collator for dynamic padding
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [18]:
from transformers import Seq2SeqTrainingArguments

# Define Seq2SeqTrainingArguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5_finetune_dgdgb-drums",  # Directory to save model checkpoints
    evaluation_strategy="epoch",            # Evaluate at the end of each epoch
    learning_rate=5e-5,                     # Learning rate
    per_device_train_batch_size=8,          # Batch size for training
    per_device_eval_batch_size=8,           # Batch size for evaluation
    num_train_epochs=3,                     # Number of epochs
    weight_decay=0.01,                      # Regularization
    save_strategy="epoch",                  # Save checkpoints after each epoch
    logging_dir="./logs",                   # Logging directory
    logging_steps=100,                      # Log every 100 steps
    save_total_limit=2,                     # Keep only the 2 most recent checkpoints
    load_best_model_at_end=True,            # Load best model at the end of training
    predict_with_generate=True,             # Generate predictions during evaluation
    fp16=True,                              # Enable mixed precision for faster training
    seed=42                                 # Random seed for reproducibility
)

# Initialize Trainer
trainer = Trainer(
    model=model,                                # T5 model
    args=training_args,                         # Training arguments
    train_dataset=tokenized_train,              # Training dataset
    eval_dataset=tokenized_validation,          # Validation dataset
    tokenizer=tokenizer,                        # Tokenizer for preprocessing
    data_collator=data_collator                 # Data collator for dynamic padding
)

<ipython-input-18-4c3d319670ab>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
# Train the model
trainer.train()

# Save the final model and tokenizer
model.save_pretrained("./t5_finetune_dgdgb-drums_final")
tokenizer.save_pretrained("./t5_finetune_dgdgb-drums_final")


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 